In [ ]:
import pandas as pd
import re
import numpy as np
import math
from textblob import TextBlob
from nltk.sentiment import SentimentIntensityAnalyzer
import operator
from spellchecker import SpellChecker
import copy
from ast import literal_eval

### Cleaning up the drugLib data

In [2]:
# Load data and combine train and test sets
drugLib_train = pd.read_csv("Data/drugLibTrain_raw.tsv", sep='\t')
drugLib_test = pd.read_csv("Data/drugLibTest_raw.tsv", sep='\t')
drugLib = pd.concat([drugLib_train, drugLib_test])

In [3]:
# Drop first column
drugLib = drugLib.drop(['Unnamed: 0'], axis=1)

# Assigning numerical values to effectiveness and sideEffects
# with a min score of 0 being ineffective or having no side effects 
# and a max score of 4 being highly effective or having extremely severe side effects.
drugLib['effectiveness'] = drugLib['effectiveness'].map({'Ineffective':0, 'Marginally Effective':1, 
                                                         'Moderately Effective':2, 'Considerably Effective':3,
                                                         'Highly Effective':4})

drugLib['sideEffects'] = drugLib['sideEffects'].map({'No Side Effects':0, 'Mild Side Effects':1, 
                                                         'Moderate Side Effects':2, 'Severe Side Effects':3,
                                                         'Extremely Severe Side Effects':4})

drugLib

,urlDrugName,rating,effectiveness,sideEffects,condition,benefitsReview,sideEffectsReview,commentsReview
0,enalapril,4,4,1,management of congestive heart failure,slowed the progression of left ventricular dys...,"cough, hypotension , proteinuria, impotence , ...","monitor blood pressure , weight and asses for ..."
1,ortho-tri-cyclen,1,4,3,birth prevention,Although this type of birth control has more c...,"Heavy Cycle, Cramps, Hot Flashes, Fatigue, Lon...","I Hate This Birth Control, I Would Not Suggest..."
2,ponstel,10,4,0,menstrual cramps,I was used to having cramps so badly that they...,Heavier bleeding and clotting than normal.,I took 2 pills at the onset of my menstrual cr...
3,prilosec,3,1,1,acid reflux,The acid reflux went away for a few months aft...,"Constipation, dry mouth and some mild dizzines...",I was given Prilosec prescription at a dose of...
4,lyrica,2,1,3,fibromyalgia,I think that the Lyrica was starting to help w...,I felt extremely drugged and dopey. Could not...,See above
...,...,...,...,...,...,...,...,...
1031,accutane,7,3,3,acne vulgaris,Detoxing effect by pushing out the system thro...,"Hairloss, extreme dry skin, itchiness, raises ...",Treatment period is 3 months/12 weeks. Dosage ...
1032,proair-hfa,10,4,0,asthma,"The albuterol relieved the constriction, irrit...",I have experienced no side effects.,I use the albuterol as needed because of aller...
1033,accutane,8,3,2,serve acne,Serve Acne has turned to middle,"Painfull muscles, problems with seeing at night","This drug is highly teratogenic ,females must ..."
1034,divigel,10,4,0,menopause,"My overall mood, sense of well being, energy l...",No side effects of any kind were noted or appa...,Divigel is a topically applied Bio-Identical H...


In [4]:
# Group by drug name to get the average rating, effectiveness, and sideEffects,
# as well as a list of all conditions, benefitsReview, sideEffectsReview, and commentsReview
# for each distinct drug.
drugLib_combined = drugLib.groupby('urlDrugName').agg(rating=('rating','mean'),
                                                      effectiveness=('effectiveness','mean'),
                                                      sideEffects=('sideEffects','mean'),
                                                      condition=('condition', list),
                                                      benefitsReview=('benefitsReview', list),
                                                      sideEffectsReview=('sideEffectsReview', list),
                                                      commentsReview=('commentsReview', list)).reset_index()
drugLib_combined

,urlDrugName,rating,effectiveness,sideEffects,condition,benefitsReview,sideEffectsReview,commentsReview
0,abilify,5.375000,2.625000,1.500000,[depression not resolved with antidepressant d...,[While on abilify I can honestly say the depre...,[but it caused memory loss and again an incide...,[I am only taking ativan & getting psychologic...
1,acanya,9.000000,4.000000,1.000000,[cystic acne],"[Acanya Gel treated cystic acne, blackheads, w...","[Dryness, slight redness and a small amount of...",[The combination of clindamycin and benzoyl pe...
2,accolate,1.000000,0.000000,0.000000,[capsuar contracture of breast implant],"[The benefits were not good, and this drug was...",[I tolerated the medication very well. I didn...,[I was required to take Accolate for three mon...
3,accupril,9.000000,3.000000,0.000000,[hypertension],[lower blood pressure in combination with Hyzaar],[None I am aware of.],[Taken once daily in combination with Hyzaar. ...
4,accutane,7.727273,3.522727,1.863636,"[acne, acne, severe acne, acne, acne, acne, ac...",[Eventually Cleared my moderate acne. Had reci...,"[Hair loss! Not so much from my head, but othe...",[I can't remember ever having the initial flar...
...,...,...,...,...,...,...,...,...
536,zyban,5.562500,2.687500,2.562500,"[smoking, stop smoking, to quit smoking, smoki...",[I stopped smoking after one week of starting ...,[There was a feeling of complete disassociatio...,"[I took one 150 mg tablet for seven days, stop..."
537,zyprexa,6.000000,2.857143,1.571429,[agitation management r/t lewy body syndrome ...,"[The Zyprexa minimized agitative outbursts, Th...","[The pt developed severe ""gait freezing"" , gai...",[Since many of these s/s are also s/s of the d...
538,zyrtec,7.781250,3.375000,1.093750,[for allergic condition and there after begnin...,[stopped sneezing runny nose and other allergi...,[my doctor toldme not to drive a car or swim w...,[i was a person with sinusitis due to allergy....
539,zyrtec-d,7.000000,3.250000,0.750000,"[allergies, hay fever, pollen, dust, mold and ...",[Zyrtec D helped me live almost symptom free e...,"[While taking Zyrtec D I often had dry mouth, ...","[Take one pill every 12 hours, although on mor..."


In [5]:
# Assign drugNameReduced column as the drug names with all special characters removed and converted to lower case
drugLib_combined['drugNameReduced'] = drugLib_combined['urlDrugName'].apply(lambda x: re.sub('[^A-Za-z0-9]', ' ', x.lower()))
drugLib_combined

,urlDrugName,rating,effectiveness,sideEffects,condition,benefitsReview,sideEffectsReview,commentsReview,drugNameReduced
0,abilify,5.375000,2.625000,1.500000,[depression not resolved with antidepressant d...,[While on abilify I can honestly say the depre...,[but it caused memory loss and again an incide...,[I am only taking ativan & getting psychologic...,abilify
1,acanya,9.000000,4.000000,1.000000,[cystic acne],"[Acanya Gel treated cystic acne, blackheads, w...","[Dryness, slight redness and a small amount of...",[The combination of clindamycin and benzoyl pe...,acanya
2,accolate,1.000000,0.000000,0.000000,[capsuar contracture of breast implant],"[The benefits were not good, and this drug was...",[I tolerated the medication very well. I didn...,[I was required to take Accolate for three mon...,accolate
3,accupril,9.000000,3.000000,0.000000,[hypertension],[lower blood pressure in combination with Hyzaar],[None I am aware of.],[Taken once daily in combination with Hyzaar. ...,accupril
4,accutane,7.727273,3.522727,1.863636,"[acne, acne, severe acne, acne, acne, acne, ac...",[Eventually Cleared my moderate acne. Had reci...,"[Hair loss! Not so much from my head, but othe...",[I can't remember ever having the initial flar...,accutane
...,...,...,...,...,...,...,...,...,...
536,zyban,5.562500,2.687500,2.562500,"[smoking, stop smoking, to quit smoking, smoki...",[I stopped smoking after one week of starting ...,[There was a feeling of complete disassociatio...,"[I took one 150 mg tablet for seven days, stop...",zyban
537,zyprexa,6.000000,2.857143,1.571429,[agitation management r/t lewy body syndrome ...,"[The Zyprexa minimized agitative outbursts, Th...","[The pt developed severe ""gait freezing"" , gai...",[Since many of these s/s are also s/s of the d...,zyprexa
538,zyrtec,7.781250,3.375000,1.093750,[for allergic condition and there after begnin...,[stopped sneezing runny nose and other allergi...,[my doctor toldme not to drive a car or swim w...,[i was a person with sinusitis due to allergy....,zyrtec
539,zyrtec-d,7.000000,3.250000,0.750000,"[allergies, hay fever, pollen, dust, mold and ...",[Zyrtec D helped me live almost symptom free e...,"[While taking Zyrtec D I often had dry mouth, ...","[Take one pill every 12 hours, although on mor...",zyrtec d


### Cleaning up the drugsCom data

In [6]:
# Load data and combine train and test sets
drugsCom_train = pd.read_csv("Data/drugsComTrain_raw.tsv", sep='\t')
drugsCom_test = pd.read_csv("Data/drugsComTest_raw.tsv", sep='\t')
drugsCom = pd.concat([drugsCom_train, drugsCom_test])
drugsCom

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37
...,...,...,...,...,...,...,...
53761,159999,Tamoxifen,"Breast Cancer, Prevention","""I have taken Tamoxifen for 5 years. Side effe...",10.0,"September 13, 2014",43
53762,140714,Escitalopram,Anxiety,"""I&#039;ve been taking Lexapro (escitaploprgra...",9.0,"October 8, 2016",11
53763,130945,Levonorgestrel,Birth Control,"""I&#039;m married, 34 years old and I have no ...",8.0,"November 15, 2010",7
53764,47656,Tapentadol,Pain,"""I was prescribed Nucynta for severe neck/shou...",1.0,"November 28, 2011",20


In [7]:
# Drop first column
drugsCom = drugsCom.drop(['Unnamed: 0', 'date'], axis=1)
drugsCom

,drugName,condition,review,rating,usefulCount
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,27
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,192
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,17
3,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,10
4,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,37
...,...,...,...,...,...
53761,Tamoxifen,"Breast Cancer, Prevention","""I have taken Tamoxifen for 5 years. Side effe...",10.0,43
53762,Escitalopram,Anxiety,"""I&#039;ve been taking Lexapro (escitaploprgra...",9.0,11
53763,Levonorgestrel,Birth Control,"""I&#039;m married, 34 years old and I have no ...",8.0,7
53764,Tapentadol,Pain,"""I was prescribed Nucynta for severe neck/shou...",1.0,20


In [8]:
# Group by drug name to get the average rating and usefulCount
# as well as a list of all conditions and reviews
# for each distinct drug.
drugsCom_combined = drugsCom.groupby('drugName').agg(rating=('rating','mean'),
                                                      usefulCount=('usefulCount','sum'),
                                                      condition=('condition', list),
                                                      review=('review', list)).reset_index()
drugsCom_combined

,drugName,rating,usefulCount,condition,review
0,A + D Cracked Skin Relief,10.000000,6,[Bacterial Skin Infection],"[""I have severe cracked skin on my hands. I&#..."
1,A / B Otic,10.000000,38,"[Otitis Media, Otitis Media]","[""It numbs the pain. It makes my ear feel heav..."
2,Abacavir / dolutegravir / lamivudine,8.414286,901,"[HIV Infection, HIV Infection, HIV Infection, ...","[""I have only been on it for 3 days now after ..."
3,Abacavir / lamivudine,10.000000,11,"[HIV Infection, HIV Infection, HIV Infection]","[""I&#039;ve used this since first diagnosed an..."
4,Abacavir / lamivudine / zidovudine,9.000000,1,[HIV Infection],"[""I was diagnosed back in 2008 and have been o..."
...,...,...,...,...,...
3666,ZzzQuil,2.000000,21,"[Insomnia, Insomnia, Insomnia]","[""I purchased the zzzquil and it did not work ..."
3667,depo-subQ provera 104,5.500000,1,"[Birth Control, Birth Control]","[""Not only did I gain ten pounds in just a few..."
3668,ella,7.183099,677,"[Emergency Contraception, Emergency Contracept...","[""Hey,\r\r\nSo I took the ellaOne about 7h aft..."
3669,femhrt,6.500000,81,"[Postmenopausal Symptoms, 0</span> users found...","[""This medication completely changed my life f..."


In [9]:
# Assign drugNameReduced column as the drug names with all special characters removed and converted to lower case
drugsCom_combined['drugNameReduced'] = drugsCom_combined['drugName'].apply(lambda x: re.sub('[^A-Za-z0-9]', ' ', x.lower()))
drugsCom_combined

,drugName,rating,usefulCount,condition,review,drugNameReduced
0,A + D Cracked Skin Relief,10.000000,6,[Bacterial Skin Infection],"[""I have severe cracked skin on my hands. I&#...",a d cracked skin relief
1,A / B Otic,10.000000,38,"[Otitis Media, Otitis Media]","[""It numbs the pain. It makes my ear feel heav...",a b otic
2,Abacavir / dolutegravir / lamivudine,8.414286,901,"[HIV Infection, HIV Infection, HIV Infection, ...","[""I have only been on it for 3 days now after ...",abacavir dolutegravir lamivudine
3,Abacavir / lamivudine,10.000000,11,"[HIV Infection, HIV Infection, HIV Infection]","[""I&#039;ve used this since first diagnosed an...",abacavir lamivudine
4,Abacavir / lamivudine / zidovudine,9.000000,1,[HIV Infection],"[""I was diagnosed back in 2008 and have been o...",abacavir lamivudine zidovudine
...,...,...,...,...,...,...
3666,ZzzQuil,2.000000,21,"[Insomnia, Insomnia, Insomnia]","[""I purchased the zzzquil and it did not work ...",zzzquil
3667,depo-subQ provera 104,5.500000,1,"[Birth Control, Birth Control]","[""Not only did I gain ten pounds in just a few...",depo subq provera 104
3668,ella,7.183099,677,"[Emergency Contraception, Emergency Contracept...","[""Hey,\r\r\nSo I took the ellaOne about 7h aft...",ella
3669,femhrt,6.500000,81,"[Postmenopausal Symptoms, 0</span> users found...","[""This medication completely changed my life f...",femhrt


### Merging the two datasets

In [10]:
# Joining the two tables using an outer join
df_combined=pd.merge(drugLib_combined,drugsCom_combined, on='drugNameReduced', how='outer')
df_combined

,urlDrugName,rating_x,effectiveness,sideEffects,condition_x,benefitsReview,sideEffectsReview,commentsReview,drugNameReduced,drugName,rating_y,usefulCount,condition_y,review
0,abilify,5.375000,2.625000,1.500000,[depression not resolved with antidepressant d...,[While on abilify I can honestly say the depre...,[but it caused memory loss and again an incide...,[I am only taking ativan & getting psychologic...,abilify,Abilify,6.443508,23756.0,"[Autism, Agitated State, Schizoaffective Disor...","[""My child has been on Abilify for a while now..."
1,acanya,9.000000,4.000000,1.000000,[cystic acne],"[Acanya Gel treated cystic acne, blackheads, w...","[Dryness, slight redness and a small amount of...",[The combination of clindamycin and benzoyl pe...,acanya,Acanya,7.345455,659.0,"[Acne, Acne, Acne, Acne, Acne, Acne, Acne, Acn...","[""Thank God i read the reviews before applying..."
2,accolate,1.000000,0.000000,0.000000,[capsuar contracture of breast implant],"[The benefits were not good, and this drug was...",[I tolerated the medication very well. I didn...,[I was required to take Accolate for three mon...,accolate,Accolate,10.000000,57.0,"[Asthma, Maintenance, Asthma, Maintenance, Ast...","[""I have had asthma all my life. About 10 yea..."
3,accupril,9.000000,3.000000,0.000000,[hypertension],[lower blood pressure in combination with Hyzaar],[None I am aware of.],[Taken once daily in combination with Hyzaar. ...,accupril,Accupril,5.000000,31.0,[Heart Failure],"[""Works well if you can tolerate the annoying ..."
4,accutane,7.727273,3.522727,1.863636,"[acne, acne, severe acne, acne, acne, acne, ac...",[Eventually Cleared my moderate acne. Had reci...,"[Hair loss! Not so much from my head, but othe...",[I can't remember ever having the initial flar...,accutane,Accutane,8.421769,10167.0,"[Acne, Acne, Acne, Acne, Acne, 13</span> users...","[""When I was a 15 year old freshman in high sc..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3722,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,zytiga,Zytiga,9.250000,423.0,"[Prostate Cance, Prostate Cance, Prostate Canc...","[""PSA 28 to 4 in one week. Fatigue only side e..."
3723,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,zzzquil,ZzzQuil,2.000000,21.0,"[Insomnia, Insomnia, Insomnia]","[""I purchased the zzzquil and it did not work ..."
3724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,depo subq provera 104,depo-subQ provera 104,5.500000,1.0,"[Birth Control, Birth Control]","[""Not only did I gain ten pounds in just a few..."
3725,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ella,ella,7.183099,677.0,"[Emergency Contraception, Emergency Contracept...","[""Hey,\r\r\nSo I took the ellaOne about 7h aft..."


##### After merging the two datasets, we combined any columns that represented the same feature into one column.

In [11]:
# Functions for combining columns that represent the same type of feature

def rating (row):
    """Returns the average rating of rating_x and rating_y if they both have existing values.
    Otherwise, only returns the one that is not NaN"""
    
    # If the rating_x value of the given row is NaN, return the value of rating_y
    if math.isnan(row['rating_x']):
        return row['rating_y']
    # If the rating_y value of the given row is NaN, return the value of rating_x
    elif math.isnan(row['rating_y']):
        return row['rating_x']
    # If both rating_x and rating_y have existing values, return the average
    else:
        return (row['rating_x'] + row['rating_y'])/2
    
def condition (row):
    """Combine condition_x and condition_y values into one list"""
    
    if isinstance(row['condition_x'], list):
        # If the condition_x and condition_y values of the given row are both lists, 
        # return their values as one list
        if isinstance(row['condition_y'], list):
            return row['condition_x'] + row['condition_y']
        # If the condition_x value is a list, but not the condition_y value, return the condition_x value
        else:
            return row['condition_x'] 
    # If the condition_y value is a list, but not the condition_x value, return the condition_y value
    else:
        return row['condition_y']
    
def review (row):
    """Combine commentsReview and review values into one list"""
    if isinstance(row['commentsReview'], list):
        # If the commentsReview and review values of the given row are both lists, 
        # return their values as one list
        if isinstance(row['review'], list):
            return row['commentsReview'] + row['review']
        # If the commentsReview value is a list, but not the review value, return the commentsReview
        else:
            return row['commentsReview']   
    # If the review value is a list, but not the commentsReview value, return the review value
    else:
        return row['review']

In [12]:
# Create new rating column of average rating_x and rating_y values
df_combined['rating'] = df_combined.apply (lambda row: rating(row), axis=1)

# Create new condition column of concatenated condition_x and condition_y values
df_combined['condition'] = df_combined.apply (lambda row: condition(row), axis=1)

# Create new reviews column of concatenated commentsReview and review values
df_combined['reviews'] = df_combined.apply (lambda row: review(row), axis=1)

df_combined

,urlDrugName,rating_x,effectiveness,sideEffects,condition_x,benefitsReview,sideEffectsReview,commentsReview,drugNameReduced,drugName,rating_y,usefulCount,condition_y,review,rating,condition,reviews
0,abilify,5.375000,2.625000,1.500000,[depression not resolved with antidepressant d...,[While on abilify I can honestly say the depre...,[but it caused memory loss and again an incide...,[I am only taking ativan & getting psychologic...,abilify,Abilify,6.443508,23756.0,"[Autism, Agitated State, Schizoaffective Disor...","[""My child has been on Abilify for a while now...",5.909254,[depression not resolved with antidepressant d...,[I am only taking ativan & getting psychologic...
1,acanya,9.000000,4.000000,1.000000,[cystic acne],"[Acanya Gel treated cystic acne, blackheads, w...","[Dryness, slight redness and a small amount of...",[The combination of clindamycin and benzoyl pe...,acanya,Acanya,7.345455,659.0,"[Acne, Acne, Acne, Acne, Acne, Acne, Acne, Acn...","[""Thank God i read the reviews before applying...",8.172727,"[cystic acne, Acne, Acne, Acne, Acne, Acne, Ac...",[The combination of clindamycin and benzoyl pe...
2,accolate,1.000000,0.000000,0.000000,[capsuar contracture of breast implant],"[The benefits were not good, and this drug was...",[I tolerated the medication very well. I didn...,[I was required to take Accolate for three mon...,accolate,Accolate,10.000000,57.0,"[Asthma, Maintenance, Asthma, Maintenance, Ast...","[""I have had asthma all my life. About 10 yea...",5.500000,"[capsuar contracture of breast implant, Asthma...",[I was required to take Accolate for three mon...
3,accupril,9.000000,3.000000,0.000000,[hypertension],[lower blood pressure in combination with Hyzaar],[None I am aware of.],[Taken once daily in combination with Hyzaar. ...,accupril,Accupril,5.000000,31.0,[Heart Failure],"[""Works well if you can tolerate the annoying ...",7.000000,"[hypertension, Heart Failure]",[Taken once daily in combination with Hyzaar. ...
4,accutane,7.727273,3.522727,1.863636,"[acne, acne, severe acne, acne, acne, acne, ac...",[Eventually Cleared my moderate acne. Had reci...,"[Hair loss! Not so much from my head, but othe...",[I can't remember ever having the initial flar...,accutane,Accutane,8.421769,10167.0,"[Acne, Acne, Acne, Acne, Acne, 13</span> users...","[""When I was a 15 year old freshman in high sc...",8.074521,"[acne, acne, severe acne, acne, acne, acne, ac...",[I can't remember ever having the initial flar...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3722,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,zytiga,Zytiga,9.250000,423.0,"[Prostate Cance, Prostate Cance, Prostate Canc...","[""PSA 28 to 4 in one week. Fatigue only side e...",9.250000,"[Prostate Cance, Prostate Cance, Prostate Canc...","[""PSA 28 to 4 in one week. Fatigue only side e..."
3723,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,zzzquil,ZzzQuil,2.000000,21.0,"[Insomnia, Insomnia, Insomnia]","[""I purchased the zzzquil and it did not work ...",2.000000,"[Insomnia, Insomnia, Insomnia]","[""I purchased the zzzquil and it did not work ..."
3724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,depo subq provera 104,depo-subQ provera 104,5.500000,1.0,"[Birth Control, Birth Control]","[""Not only did I gain ten pounds in just a few...",5.500000,"[Birth Control, Birth Control]","[""Not only did I gain ten pounds in just a few..."
3725,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ella,ella,7.183099,677.0,"[Emergency Contraception, Emergency Contracept...","[""Hey,\r\r\nSo I took the ellaOne about 7h aft...",7.183099,"[Emergency Contraception, Emergency Contracept...","[""Hey,\r\r\nSo I took the ellaOne about 7h aft..."


##### To convert reviews into numerical values, we perfomed sentiment analysis on those columns to get a sentiment score.

In [13]:
def sentiment_analysis(review_list):
    """Performs sentiment analysis on every review in the given list. Then, takes the average of each sentiment
    score. If there are no given reviews to analyze, automatically assigns score of 0.
    A negative score represents a negative review, a score of 0 represents a neutral review, and 
    a positive score represents a positive review."""
    
    sia = SentimentIntensityAnalyzer()
    review_scores = []
    
    if isinstance(review_list, list):
        # Iterate through each review and assign a sentiment score
        # Then, return the average of all the sentiment scores
        for review in review_list:
            score = sia.polarity_scores(str(review))["compound"]
            review_scores.append(score)
        return np.mean(review_scores)
    
    # If the given object is not a list of reviews, return a score of 0
    return 0

In [14]:
# Sentiment analysis on the review columns
df_combined["sentiment_score"] = df_combined["reviews"].apply(lambda x: sentiment_analysis(x))
df_combined["benefits_sentiment_score"] = df_combined["benefitsReview"].apply(lambda x: sentiment_analysis(x))
df_combined["side_effects_sentiment_score"] = df_combined["sideEffectsReview"].apply(lambda x: sentiment_analysis(x))

# Drop columns we no longer need
df_combined = df_combined.drop(['rating_x', 'rating_y', 'condition_x', 'condition_y', 'commentsReview', 
                                'review', 'reviews', 'benefitsReview', 'sideEffectsReview'], axis=1)

In [15]:
df_combined

,urlDrugName,effectiveness,sideEffects,drugNameReduced,drugName,usefulCount,rating,condition,sentiment_score,benefits_sentiment_score,side_effects_sentiment_score
0,abilify,2.625000,1.500000,abilify,Abilify,23756.0,5.909254,[depression not resolved with antidepressant d...,0.019727,0.254600,-0.264713
1,acanya,4.000000,1.000000,acanya,Acanya,659.0,8.172727,"[cystic acne, Acne, Acne, Acne, Acne, Acne, Ac...",0.248393,0.126300,0.000000
2,accolate,0.000000,0.000000,accolate,Accolate,57.0,5.500000,"[capsuar contracture of breast implant, Asthma...",0.465575,-0.286900,0.564800
3,accupril,3.000000,0.000000,accupril,Accupril,31.0,7.000000,"[hypertension, Heart Failure]",0.259300,-0.526700,0.000000
4,accutane,3.522727,1.863636,accutane,Accutane,10167.0,8.074521,"[acne, acne, severe acne, acne, acne, acne, ac...",0.167362,0.172561,-0.189648
...,...,...,...,...,...,...,...,...,...,...,...
3722,NaN,NaN,NaN,zytiga,Zytiga,423.0,9.250000,"[Prostate Cance, Prostate Cance, Prostate Canc...",-0.132458,0.000000,0.000000
3723,NaN,NaN,NaN,zzzquil,ZzzQuil,21.0,2.000000,"[Insomnia, Insomnia, Insomnia]",0.208200,0.000000,0.000000
3724,NaN,NaN,NaN,depo subq provera 104,depo-subQ provera 104,1.0,5.500000,"[Birth Control, Birth Control]",0.255600,0.000000,0.000000
3725,NaN,NaN,NaN,ella,ella,677.0,7.183099,"[Emergency Contraception, Emergency Contracept...",-0.354651,0.000000,0.000000


##### For rows that had NaN values for effectiveness and/or sideEffects, we replaced the NaN values with the mean effectiveness and sideEffects scores.

In [16]:
# Calculate mean effectiveness
effectiveness = df_combined['effectiveness']
effectiveness = effectiveness.dropna()
effectiveness_mean = effectiveness.mean()

# Calculate mean sideEffects
sideEffects = df_combined['sideEffects']
sideEffects = sideEffects.dropna()
sideEffects_mean = sideEffects.mean()

# Assign Nan values to mean effectiveness/sideEffects value
df_combined['effectiveness'] = df_combined['effectiveness'].fillna(effectiveness_mean)
df_combined['sideEffects'] = df_combined['sideEffects'].fillna(sideEffects_mean)

df_combined

,urlDrugName,effectiveness,sideEffects,drugNameReduced,drugName,usefulCount,rating,condition,sentiment_score,benefits_sentiment_score,side_effects_sentiment_score
0,abilify,2.625000,1.500000,abilify,Abilify,23756.0,5.909254,[depression not resolved with antidepressant d...,0.019727,0.254600,-0.264713
1,acanya,4.000000,1.000000,acanya,Acanya,659.0,8.172727,"[cystic acne, Acne, Acne, Acne, Acne, Acne, Ac...",0.248393,0.126300,0.000000
2,accolate,0.000000,0.000000,accolate,Accolate,57.0,5.500000,"[capsuar contracture of breast implant, Asthma...",0.465575,-0.286900,0.564800
3,accupril,3.000000,0.000000,accupril,Accupril,31.0,7.000000,"[hypertension, Heart Failure]",0.259300,-0.526700,0.000000
4,accutane,3.522727,1.863636,accutane,Accutane,10167.0,8.074521,"[acne, acne, severe acne, acne, acne, acne, ac...",0.167362,0.172561,-0.189648
...,...,...,...,...,...,...,...,...,...,...,...
3722,NaN,2.933651,1.193745,zytiga,Zytiga,423.0,9.250000,"[Prostate Cance, Prostate Cance, Prostate Canc...",-0.132458,0.000000,0.000000
3723,NaN,2.933651,1.193745,zzzquil,ZzzQuil,21.0,2.000000,"[Insomnia, Insomnia, Insomnia]",0.208200,0.000000,0.000000
3724,NaN,2.933651,1.193745,depo subq provera 104,depo-subQ provera 104,1.0,5.500000,"[Birth Control, Birth Control]",0.255600,0.000000,0.000000
3725,NaN,2.933651,1.193745,ella,ella,677.0,7.183099,"[Emergency Contraception, Emergency Contracept...",-0.354651,0.000000,0.000000


##### For the conditions column, we cleaned up each list of conditions by fixing spelling errors and removing duplicate conditions.

In [19]:
def reduce_condition(condition_list):
    """Clean up the given list of conditions fix spelling errors, make all conditions lower case,
    and remove redundant and irrelevant values.
    """
    # Remove all floats
    conditions = [b for b in condition_list if not isinstance(b, float)]
    
    # Iterate through each condition to clean up spelling
    for condition in conditions:
        # make lowercase and fix double spaces
        condition_lc = str(condition).lower().replace("  ", " ")
        # Use TextBlob to fix spelling errors
        gfg = TextBlob(condition_lc)
        gfg = gfg.correct()
        conditions[conditions.index(condition)] = str(gfg)
        
    # remove items that contain '<' or '/'
    conditions = [x for x in conditions if (not '<' in x and not '/' in x)]
    # split by 'and'
    conditions = [y for x in conditions for y in x.split(' and ')]
    
    # Convert the list to a set to remove redundant values
    cond_set = set(conditions)
        
    return list(cond_set)

In [ ]:
# Reduce the conditions column and save as reduced_conditions column
df_combined['reduced_conditions'] = df_combined['condition'].apply(lambda x: reduce_condition(x))
df_combined

##### Saving data as csv files to load into Neo4J

In [ ]:
# Save as csv to load as Drug nodes in Neo4J
df_combined.to_csv('/csv/drug_info.csv')

In [13]:
# Get set of all unique conditions
condition_set = set()
for row in range(3723):
    condition_list = literal_eval(str(df_combined['reduced_conditions'][row]))
    condition_set.update(condition_list)
    
# Store conditions in a DataFrame and save as csv file
df_conditions = pd.DataFrame(list(condition_set), columns=['condition'])

# Save as csv to load as Condition nodes in Neo4J
df_conditions.to_csv('/csv/conditions.csv')

{'chronic idiopathic urticaria with neuritis',
 'extreme hot flashes',
 'red spot on the cheek',
 'menopause',
 'diabetes, type 2',
 'urgent privation',
 'high deficiency & his',
 'ascariasis',
 'pityriasis cuba paris',
 'gonococcal infection, uncomplicated',
 'malign glioma',
 'dystonia',
 'chronic depression',
 'anti-raging',
 'polycystic ovarian syndrome',
 'lupus-like system autoimmune condition',
 'multilocular goitre',
 'opiates dependency',
 'double ear infections',
 'cocaine dependency',
 'spring galleries, clergy cough',
 'temporomandibular joint disorder (tm)',
 'dissociative identity disorder',
 'impetigo',
 'bipolar disorder, old behavior',
 'to help me sleep after having a rapid heartfelt up',
 'primary hypersomnia',
 'least infections due to dryness',
 'interactive thyreoid',
 'submental fullness',
 'sleeplessness - depression',
 'antibiotic to treat oral infection',
 'there after begging of',
 'conjunctivitis, bacterial',
 'acute ostitis externa',
 'nicotine addition',
 

In [19]:
# Get dataframe of all drug-condition pairs
drug = []
condition = []

# Iterate through each row to extract each drug and its list of conditions
for row in range(3723):
    condition_list = literal_eval(str(df_combined['reduced_conditions'][row]))
    
    # Iterate through the list of conditions to keep track of each drug-condition pair for the current row
    for cond in condition_list:
        drug.append(df_combined['drugNameReduced'][row])
        condition.append(cond)

# Add list of drugs and their corresponding conditions to a DataFrame
df_treatment = pd.DataFrame(list(zip(drug, condition)),columns =['drug', 'condition'])

# Save as csv to load as Drug-Condition edges in Neo4J
df_treatment.to_csv('/csv/treatment.csv')  

### Combining with GAD dataset

In [2]:
# Load our list of unique conditions
conditions = pd.read_csv("/csv/conditions.csv", index_col=0)
conditions

,condition
0,chronic idiopathic urticaria with neuritis
1,extreme hot flashes
2,red spot on the cheek
3,menopause
4,"diabetes, type 2"
...,...
2218,inability to fall asleep easily
2219,dandruff
2220,acute coronary syndrome
2221,adult moderate harmony related acne


In [3]:
# Load GAD dataset
gad = pd.read_csv("Data/gad.csv")
# Replace the 'hypertension' phenotype with 'high blood pressure' to better match our drug-condition dataset
gad['phenotype'] = gad['phenotype'].replace('hypertension', 'high blood pressure')
# Reduce phenotypes by removing special characters and converting to lowercase
gad['phenotype_reduced'] = gad['phenotype'].apply(lambda x: re.sub('[^A-Za-z0-9]', ' ', str(x).lower()))
# Drop unecessary columns
gad = gad.drop(['reference', 'pubmed_id', 'year', 'population', 'gad_id'], axis=1)
# Only keep the rows that do have a phenotype-gene association
gad = gad[gad['association'] == 'Y']
# Drop duplicate rows
gad = gad.drop_duplicates()
# Replace NaN values with zeros
gad['chromosome'] = gad['chromosome'].fillna(0)
gad['chromosome_band'] = gad['chromosome_band'].fillna(0)
gad['gene_name'] = gad['gene_name'].fillna(0)

gad

,association,phenotype,disease_class,chromosome,chromosome_band,dna_start,dna_end,gene,gene_name,phenotype_reduced
2,Y,Parkinson's Disease,NEUROLOGICAL,12,12p13.3-p12.3,9111570,9159825,A2M,Alpha-2-macroglobulin,parkinson s disease
9,Y,Alzheimer`s disease,NEUROLOGICAL,12,12p13.3-p12.3,9111570,9159825,A2M,Alpha-2-macroglobulin,alzheimer s disease
15,Y,normal variation,NORMALVARIATION,12,12p13.3-p12.3,9111570,9159825,A2M,Alpha-2-macroglobulin,normal variation
16,Y,rheumatoid arthritis,IMMUNE,12,12p13.3-p12.3,9111570,9159825,A2M,Alpha-2-macroglobulin,rheumatoid arthritis
20,Y,argyrophilic grain disease,NEUROLOGICAL,12,12p13.3-p12.3,9111570,9159825,A2M,Alpha-2-macroglobulin,argyrophilic grain disease
...,...,...,...,...,...,...,...,...,...,...
39904,Y,asthma eczema,IMMUNE,1,1q21,150541275,150564303,FLG,Filaggrin,asthma eczema
39905,Y,eczema,IMMUNE,1,1q21,150541275,150564303,FLG,Filaggrin,eczema
39907,Y,venous thrombosis,CARDIOVASCULAR,4,4q35.2,187487823,187509760,CYP4V2,"cytochrome P450, family 4, subfamily V, polype...",venous thrombosis
39908,Y,venous thrombosis,CARDIOVASCULAR,1,1q23-q25.1,172139564,172153096,SERPINC1,"serpin peptidase inhibitor, clade C (antithrom...",venous thrombosis


In [4]:
# Inner join of the conditions and gad dataframes, matching on condition/phenotype names
condition_gad = conditions.merge(gad, left_on='condition', right_on='phenotype_reduced', how='inner')

# Save as csv to load as Condition-Gene edges in Neo4J
condition_gad.to_csv('/csv/gene_interactions.csv')

In [5]:
# Get dataframe of unique genes

# Drop duplicate rows
gad_unique = condition_gad.drop_duplicates(['chromosome','chromosome_band', 'dna_start', 
                                            'dna_end', 'gene'],keep= 'last')

# Only keep the chromosome, chromosome_band, dna_start, dna_end, gene, and gene_name columns
gad_unique = gad_unique[['chromosome','chromosome_band', 'dna_start', 
                                            'dna_end', 'gene', 'gene_name']]

# Save as csv to lead as Gene nodes in Neo4J
gad_unique.to_csv('/csv/genes.csv')